# Cleaning up the raw data

This script exists to do three things

1. Expand the filename dummies into columns
2. Drop duplicate time indexes
3. Combine all raw files into one feather file

The Arrow parquet format will store our data more accurately, unlike csv. It is also much smaller.

(Support for feather improves with pandas 1.1.0, in particular compression: https://arrow.apache.org/docs/python/generated/pyarrow.feather.write_feather.html)

### Imports

In [1]:
import pandas as pd
import os
import re

### Importing all raw files

The data logging script will output files with prdictabhle filenames, so we can use regular expressions to safely import them all.

The code below will find all files in the `./data` sub-folder. With the `re.match()` regular expression, we can only return files that start with `pistress` but that are also `.csv`.

#### Regex explanation

I used a raw string `r""` to make regex easier to write. Normally if you want to use a backslah `\` you have to type `\\` to espace it first.

The `^` character will match the start of the filename, and the `$` character will match the end of the filename. These are great for making sure you're really getting what you're expecting. For example, we wouldn't want a file ending in `.csview`.

The `\d{10}` regular expression will match exactly 10 digit characters. Our code below expects 10-digit dummies, so we should be strict with our inputs. **If you add more dummies, you will have to change the amount of character digits here.**

In [2]:
raw_files = [x for x in os.listdir("./data") if re.match(r"^pistress_\d{10}.csv$", x)]

raw_files

['pistress_1110100011.csv',
 'pistress_1111000111.csv',
 'pistress_1111010011.csv',
 'pistress_1110001011.csv',
 'pistress_1100000011.csv',
 'pistress_1101000011.csv',
 'pistress_1000000011.csv',
 'pistress_1100000111.csv',
 'pistress_1101000111.csv',
 'pistress_1100100011.csv',
 'pistress_1101010011.csv',
 'pistress_0000000011.csv',
 'pistress_1100010011.csv',
 'pistress_1100001011.csv',
 'pistress_1111001011.csv',
 'pistress_1101100011.csv',
 'pistress_1111100011.csv']

#### Reading into pandas

We can send each filename to the `pd.read_csv()` function and then "chain" `.assign()` to add in the filename as its own column.

The list of these dataframes can then be given to `pd.concat()` for merging into one big dataframe.

In [3]:
raw = pd.concat([pd.read_csv(f"./data/{x}").assign(filename=x) for x in raw_files])

raw

,datetime,usage,temp,stress,load,filename
0,2020-07-30 17:51:07.023100,44.5,46.698,2,NaN,pistress_1110100011.csv
1,2020-07-30 17:51:08.025728,49.9,47.236,2,NaN,pistress_1110100011.csv
2,2020-07-30 17:51:09.027797,50.0,48.312,2,NaN,pistress_1110100011.csv
3,2020-07-30 17:51:10.029807,50.0,48.312,2,NaN,pistress_1110100011.csv
4,2020-07-30 17:51:11.031788,50.2,48.850,2,NaN,pistress_1110100011.csv
...,...,...,...,...,...,...
3587,2020-07-30 08:54:52.443320,25.0,78.440,1,NaN,pistress_1111100011.csv
3588,2020-07-30 08:54:53.445236,25.1,78.440,1,NaN,pistress_1111100011.csv
3589,2020-07-30 08:54:54.447141,25.4,77.902,1,NaN,pistress_1111100011.csv
3590,2020-07-30 08:54:55.449056,25.0,77.902,1,NaN,pistress_1111100011.csv


### Creating dummy variables

You may know these as indicator variables or as flags.

This information has been embedded into the filenames. Here is the significance of the filename codes.

1. (a) case_under
2. (b) case_frame
3. (c) case_cable
4. (d) case_gpio
5. (m) top_solid
6. (n) top_holed
7. (o) top_intake (fan)
8. (p) top_exhaust (fan)
9. (x) heatsink_main
10. (y) heatsink_sub

For example, `1111000111` is a fully formed case with an exhaist fan and heatsinks on both ICs.

#### Dummification

The code below will find all unique filenames and encode them. These can then be joined to the raw data with the common `filename` key.

In [4]:
def dummify_filename(filename):
    # The filenames have an underscore
    # Split on _ and keep the second half
    code = filename.split("_")[1]
    # The string will still have .csv at the end
    # Split on . and keep the first half
    code = filename.split(".")[0]
    # Get rid of non-digit characters
    code = re.sub(r"\D", "", code)
    # Break the code into a list of 0/1 integers
    flags = [int(x) for x in list(code)]
    # List the dummy labels
    keys = ["case_under",
            "case_frame",
            "case_cable",
            "case_gpio",
            "top_solid",
            "top_holed",
            "top_intake",
            "top_exhaust",
            "heatsink_main",
            "heatsink_sub"
           ]
    # Output into a dictionary, which pandas can transform into a dataframe
    values = {k:v for k,v in zip(keys, flags)}
    values["filename"] = filename
    return values

# Only get unique filenames
# Feed them into the dummify function
flags = pd.DataFrame([dummify_filename(x) for x in raw.filename.drop_duplicates()])

flags

,case_under,case_frame,case_cable,case_gpio,top_solid,top_holed,top_intake,top_exhaust,heatsink_main,heatsink_sub,filename
0,1,1,1,0,1,0,0,0,1,1,pistress_1110100011.csv
1,1,1,1,1,0,0,0,1,1,1,pistress_1111000111.csv
2,1,1,1,1,0,1,0,0,1,1,pistress_1111010011.csv
3,1,1,1,0,0,0,1,0,1,1,pistress_1110001011.csv
4,1,1,0,0,0,0,0,0,1,1,pistress_1100000011.csv
5,1,1,0,1,0,0,0,0,1,1,pistress_1101000011.csv
6,1,0,0,0,0,0,0,0,1,1,pistress_1000000011.csv
7,1,1,0,0,0,0,0,1,1,1,pistress_1100000111.csv
8,1,1,0,1,0,0,0,1,1,1,pistress_1101000111.csv
9,1,1,0,0,1,0,0,0,1,1,pistress_1100100011.csv


#### Merging

The two dataframes can be merged together, adding the column dummy variables. The `filename` column can be dropped since its information content has been extracted.

In [5]:
# The .join() method joins on indexes
df_flagged = raw.merge(flags, on = "filename")

# .drop() the filename, with axis=1 for columns
df_flagged = df_flagged.drop("filename", axis=1)

df_flagged

,datetime,usage,temp,stress,load,case_under,case_frame,case_cable,case_gpio,top_solid,top_holed,top_intake,top_exhaust,heatsink_main,heatsink_sub
0,2020-07-30 17:51:07.023100,44.5,46.698,2,NaN,1,1,1,0,1,0,0,0,1,1
1,2020-07-30 17:51:08.025728,49.9,47.236,2,NaN,1,1,1,0,1,0,0,0,1,1
2,2020-07-30 17:51:09.027797,50.0,48.312,2,NaN,1,1,1,0,1,0,0,0,1,1
3,2020-07-30 17:51:10.029807,50.0,48.312,2,NaN,1,1,1,0,1,0,0,0,1,1
4,2020-07-30 17:51:11.031788,50.2,48.850,2,NaN,1,1,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61053,2020-07-30 08:54:52.443320,25.0,78.440,1,NaN,1,1,1,1,1,0,0,0,1,1
61054,2020-07-30 08:54:53.445236,25.1,78.440,1,NaN,1,1,1,1,1,0,0,0,1,1
61055,2020-07-30 08:54:54.447141,25.4,77.902,1,NaN,1,1,1,1,1,0,0,0,1,1
61056,2020-07-30 08:54:55.449056,25.0,77.902,1,NaN,1,1,1,1,1,0,0,0,1,1


### Creating the datetime index

We're better off with a datetime index. The `pandas` documentation always talks about it, so I get the impression it's a best practice.

There is a practical benefit to the `DateTimeIndex` too. We can use the `.round()` method to round the datetimes to the second, dropping the extra precision we don't really need. The temperature logger read sensors every second, so the sub-second time scale is not really informative.

In [6]:
df_dt = df_flagged

# Convert the datetime column to a datetime dtype
# This works fine since the datetime is already in a standard format
df_dt["datetime"] = pd.to_datetime(df_dt["datetime"])
df_dt["datetime"] = pd.DatetimeIndex(df_dt["datetime"]).round("s")

# Set datetime as the index
df_dt = df_dt.set_index("datetime")

df_dt

,usage,temp,stress,load,case_under,case_frame,case_cable,case_gpio,top_solid,top_holed,top_intake,top_exhaust,heatsink_main,heatsink_sub
datetime,,,,,,,,,,,,,,
2020-07-30 17:51:07,44.5,46.698,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:08,49.9,47.236,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:09,50.0,48.312,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:10,50.0,48.312,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:11,50.2,48.850,2,NaN,1,1,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-30 08:54:52,25.0,78.440,1,NaN,1,1,1,1,1,0,0,0,1,1
2020-07-30 08:54:53,25.1,78.440,1,NaN,1,1,1,1,1,0,0,0,1,1
2020-07-30 08:54:54,25.4,77.902,1,NaN,1,1,1,1,1,0,0,0,1,1


### Drop any duplicate indexes

The data should not have any observations in the same second. The delay between sensor readings should be at least 1 second.

Nevertheless I will detect and remove duplicates. I almost always do this when working with data. Duplicates have burned me in the past!

In [7]:
# Do we have any duplicated indexes?
# It's a good idea to include this printout for information purposes
print(f"We have duplicate indexes: {any(df_dt.index.duplicated())}")

We have duplicate indexes: False


In [8]:
# Drop duplicate indexes, just in case
# I learned something: the tilde sign in Python is bitwise not
# ie: "vectorized" not if you're from R
df = df_dt.loc[~df_dt.index.duplicated()]

df

,usage,temp,stress,load,case_under,case_frame,case_cable,case_gpio,top_solid,top_holed,top_intake,top_exhaust,heatsink_main,heatsink_sub
datetime,,,,,,,,,,,,,,
2020-07-30 17:51:07,44.5,46.698,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:08,49.9,47.236,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:09,50.0,48.312,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:10,50.0,48.312,2,NaN,1,1,1,0,1,0,0,0,1,1
2020-07-30 17:51:11,50.2,48.850,2,NaN,1,1,1,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-30 08:54:52,25.0,78.440,1,NaN,1,1,1,1,1,0,0,0,1,1
2020-07-30 08:54:53,25.1,78.440,1,NaN,1,1,1,1,1,0,0,0,1,1
2020-07-30 08:54:54,25.4,77.902,1,NaN,1,1,1,1,1,0,0,0,1,1


### Writing to disk

Below I write the cleaned dataframe to a csv.

In [9]:
df.reset_index().to_parquet("./data/cleaned.parquet")

In [10]:
# In the near future, feather files will offer compression (pandas 1.1.0)
# df.reset_index().to_feather("./data/cleaned.feather")